In [3]:
%load_ext autoreload
%autoreload 2

import os 
from pathlib import Path

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def mkdir(path: Path) -> Path:
    path = Path(path)
    if path.suffix != '':
        path = path.parent
    if path.exists():
        print('path exists, leaving alone')
    else:
        path.mkdir(parents=True)
    return path

this_dir = Path('').parent
TMP = mkdir('./tmp/out')

path exists, leaving alone


In [4]:
import jax
from jax import numpy as jnp
from PIL import Image

from flax import jax_utils
from flax.training import common_utils, train_state, dynamic_scale
import optax

def flatten(x):
  return x.reshape(x.shape[0], -1)

def trs(b):
    return jnp.array(b.numpy())[None]

In [5]:
import paramiko
import sys
import subprocess
import wandb
from time import sleep
from functools import partial, reduce
from itertools import product
from simple_slurm import Slurm
import random
from typing import Any, Iterable
import re
from ast import literal_eval
import jax
from jax import numpy as jnp
from jax import random as rnd
from typing import Any
import optax
from flax.training.train_state import TrainState
from flax import linen as nn, jax_utils
from jax.config import config
config.update('jax_disable_jit', True)


compute_rvec = lambda x0, x1: \
        jnp.expand_dims(x0, -2) - jnp.expand_dims(x1, -3)

compute_r = lambda x0, x1, keepdims=True: \
        jnp.linalg.norm(compute_rvec(x0, x1), axis=-1, keepdims=keepdims)

l1_norm_keep = lambda x: jnp.linalg.norm(x, axis=-1, keepdims=True)

# into pyfig
def compute_terms_in(x, *, a=None, terms=['x']):
    z = []
    if 'x' in terms:
        z += [x]
    if 'x_r' in terms:
        z += [jnp.linalg.norm(x, axis=-1, keepdims=True)]
    if 'xa' in terms:
        z += [compute_rvec(x, a)]
    if 'xa_rvec' in terms:
        z += [compute_r(x, a)]
    if 'xx' in terms:
        z += [compute_r(x, x)]
    if 'xx_rvec' in terms:
        z += [compute_rvec(x, x)]
    return jnp.concatenate(z, axis=-1)



In [6]:

def get_cartesian_product(*args):
    """ Cartesian product is the ordered set of all combinations of n sets """
    return list(product(*args))

def zip_in_n_chunks(arg: Iterable[Any], n: int) -> zip:   
    return zip(*([iter(arg)]*n))

def flat_list(lst_of_lst):
    return [lst for sublst in lst_of_lst for lst in sublst]

def gen_alphanum(n: int = 7, test=False):
    from string import ascii_lowercase, ascii_uppercase
    random.seed(test if test else None)
    numbers = ''.join([str(i) for i in range(10)])
    characters = ascii_uppercase + ascii_lowercase + numbers
    name = ''.join([random.choice(characters) for _ in range(n)])
    return name

def add_to_Path(path: Path, string: str | Path):
        return Path(str(path) + str(string))

def iterate_folder(folder: Path, iter_exp_dir):
    if iter_exp_dir and folder.exists():
        for i in range(100):
            _folder = add_to_Path(folder, f'-{i}')
            if not re.search(_folder.name, f'-[0-9]*'):
                folder = _folder
                break
        else:
            folder = add_to_Path(folder, f'-0')
    return folder
    
class Sub:
    
    def __init__(_i, parent=None):
        _i.parent = parent
        _i.__safe_init__()

    def __safe_init__(_i):
        cls_d = dict(_i.__class__.__dict__)
        sub_cls = {k:v for k,v in cls_d.items() if isinstance(v, type)}
        [cls_d.pop(k) for k in sub_cls.keys()]
        for k,v in cls_d.items():
            # print(k, type(v), callable(v))
            if k.startswith('__') or k in Pyfig._ignore_attr:
                continue # this was around because 'dicts are unhashable types'
            if callable(v) or isinstance(v, property):
                continue
            setattr(_i, k, v)
        [setattr(_i, k, v(parent=_i)) for k,v in sub_cls.items()]

    @property
    def dict(_i,):
        d = cls_to_dict(_i, Pyfig._ignore_attr)
        for k,v in d.items():
            if issubclass(type(v), Sub):
                d[k] = cls_to_dict(v, Pyfig._ignore_attr)
        return d

def cls_to_dict(cls, ignore:list)->dict:
    d = {}
    for k,v in cls.__dict__.items():
        if k.startswith('_') or k in ignore:
            continue
        if callable(v):
            continue
        if issubclass(type(v), Sub):
            d[k] = cls_to_dict(v, ignore)
            continue
        d[k] = getattr(cls,k)
    return d

def cmd_to_dict(cmd:str|list,ref:dict,_d={},delim:str=' --'):
    """
    fmt: [--flag, arg, --true_flag, --flag, arg1]
    # all flags double dash because of negative numbers duh """
    booleans = ['True', 'true', 't', 'False', 'false', 'f']
    
    cmd = ' '.join(cmd) if isinstance(cmd, list) else cmd
    cmd = [x.lstrip().lstrip('--').rstrip() for x in cmd.split(delim)]
    cmd = [x.split(' ', maxsplit=1) for x in cmd if ' ' in x]
    [x.append('True') for x in cmd if len(x) == 1]
    cmd = flat_list(cmd)
    cmd = iter([x.strip() for x in cmd])

    for k,v in zip(cmd, cmd):
        if v in booleans: 
            v=booleans.index(v)<3  # 0-2 True 3-5 False
        if k in ref:
            _d[k] = type(ref[k])(v)
        else:
            try:
                _d[k] = literal_eval(v)
            except:
                _d[k] = str(v)
            print(f'Guessing type: {k} as {type(v)}')
    return _d

def update_cls_with_dict(cls: Any, d:dict):
    cls_all = [v for v in cls.__dict__.values() if issubclass(type(v), Sub)]
    cls_all.extend([cls])
    n_remain = len(d)
    for k,v in d.items():
        for _cls_assign in cls_all:            
            if not hasattr(_cls_assign, k):
                continue
            else:
                if isinstance(cls.__class__.__dict__[k], property):
                    print('Tried to assign property, consider your life choices')
                    continue
                v = type(cls.__dict__)(v)
                setattr(_cls_assign, k, v)
                n_remain -= 1
    return n_remain

def cls_to_dict(cls, ignore:list)->dict:
    return {k: getattr(cls, k) for k in dir(cls) if not (k.startswith('_') or k in ignore)}

def flat_dict(d:dict,items:list=[]):
    for k,v in d.items():
        if isinstance(v, dict):
            items.extend(flat_dict(v, items=items).items())
        else:
            items.append((k, v))
    return dict(items)  

def dict_to_wandb(d:dict,parent_key:str='',sep:str ='.',items:list=[])->dict:
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict): 
            items.extend(dict_to_wandb(v,new_key,items=items).items())
        else:
            if isinstance(v, Path):  v=str(v)
            items.append((new_key, v))
    return dict(items)

def count_gpu() -> int: # output = run_cmd('echo $CUDA_VISIBLE_DEVICES', cwd='.')
    return sum(c.isdigit() for c in os.environ.get('CUDA_VISIBLE_DEVICES'))

def run_cmds(cmd:str|list,cwd:str|Path=None,input_req:str=None):
    _out = []
    for cmd_1 in (cmd if isinstance(cmd, list) else [cmd]): 
        cmd_1 = [c.strip() for c in cmd_1.split(' ')]
        _out += [subprocess.run(
            cmd_1,cwd=cwd,input=input_req, capture_output=True)]
        sleep(0.1)
    return _out

def run_cmds_server(server:str,user:str,cmd:str|list,cwd=str|Path):
    _out = []
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # if not known host
    client.connect(hostname=server, username=user)
    client.exec_command(f'cd {cwd}')
    with client as _r:
        for cmd_1 in (cmd if isinstance(cmd, list) else [cmd]):
            _out += [_r.exec_command(f'{cmd_1}')] # in, out, err
            sleep(0.1)
    return _out

class Pyfig(Sub):

    seed:               int     = 808017424 # grr
    project_root:       str     = Path().home()/'projects'

    project:            str     = 'hwat'
    project_path:       Path    = property(lambda _: _.project_root / _.project)
    server_project_path:Path    = property(lambda _: _.project_path)
    n_device:           int     = property(lambda _: count_gpu())

    exp_name:           str     = 'junk'
    run_path:           Path    = property(lambda _: _.project_path / 'run.py')
    data_dir:           Path    = project_root / 'data'
    
    half_precision:     bool    = True
    dtype:              str     = 'f32'
    n_step:             int     = 1000

    af:                 str     = 'tanh' # activation function
    n_layer:            int     = 3
    
    class data(Sub):
        dataset     = 'fashion_mnist'
        b_size      = 16
        cache       = False
        image_size  = 28
        channels    = 1

    class model(Sub):
        dim         = 64
        dim_mults   = (1, 2, 4)

    class opt(Sub):
        optimizer   = 'Adam'
        beta1       = 0.9
        beta2       = 0.99
        eps         = 1e-8
        lr          = 0.001
        loss        = 'l1'  # change this to loss table load? 

    class sweep(Sub):
        method      = 'random'
        name        = 'sweep'
        metrics = dict(
            goal    = 'minimize',
            name    = 'validation_loss',
        )
        parameters = dict(
            batch_size  = {'values' : [16, 32, 64]},
            epoch       = {'values' : [5, 10, 15]},
            lr          = {'max'    : 0.1, 'min': 0.0001},
        )
        n_sweep = run_cap = reduce(
            lambda i,j:i*j,[len(v['values']) for k,v in parameters.items() if 'values' in v])+1
        sweep_id = ''

    class wandb(Sub):
        job_type:       str     = 'training'
        entity:         str     = 'xmax1'

    log_sample_step:    int     = 5
    log_metric_step:    int     = 5
    log_state_step:     int     = 10         # wandb entity
    n_epoch:            int     = 20

    class slurm(Sub):
        output          = TMP/'o-%j.out'
        error           = TMP/'e-%j.err'
        mail_type       = 'FAIL'
        partition       ='sm3090'
        nodes           = 1                # n_node
        ntasks          = 8                # n_cpu
        cpus_per_task   = 1     
        time            = '0-12:00:00'     # D-HH:MM:SS
        gres            = 'gpu:RTX3090:1'
        job_name        = property(lambda _: _.parent.exp_name)  # this does not call the instance it is in
        sbatch          = property(lambda _: f""" 
            module purge 
            source ~/.bashrc 
            module load GCC 
            module load CUDA/11.4.1 
            module load cuDNN/8.2.2.26-CUDA-11.4.1 
            conda activate {_.parent.env} 
            export MKL_NUM_THREADS=1 
            export NUMEXPR_NUM_THREADS=1 
            export OMP_NUM_THREADS=1 
            export OPENBLAS_NUM_THREADS=1
            pwd
            nvidia-smi
            mv_cmd = f'mv {TMP}/o-$SLURM_JOB_ID.out {TMP}/e-$SLURM_JOB_ID.err $out_dir' 
    """)

    exp_id:             str     = gen_alphanum(n=7)
    
    iter_exp_dir:       bool    = True
    project_exp_dir:    Path    = property(lambda _: _.project_path / 'exp')
    project_cfg_dir:    Path    = property(lambda _: _.project_path / 'cfg')
    exp_path:           Path    = property(lambda _: iterate_folder(_.project_exp_dir/_.exp_name,_.iter_exp_dir)/_.exp_id)

    server:             str     = 'svol.fysik.dtu.dk'   # SERVER
    user:               str     = 'amawi'     # SERVER
    entity:             str     = 'xmax1'       # WANDB entity
    git_remote:         str     = 'origin'      
    git_branch:         str     = 'main'        
    env:                str     = 'dex'            # CONDA ENV
    commit_id:          str     = property(lambda _: _.get_commit_id())
    
    _sys_arg: list = sys.argv[1:]
    _submit_state:     int     = -1
    _ignore_attr = ['parent','protected','dict','cmd']

    def __init__(_i,args:dict={},cap=40,wandb_mode='online',notebook=False):
        super().__init__()
        _i.__safe_init__()

        update_cls_with_dict(_i,args)
        if not notebook:
            update_cls_with_dict(cmd_to_dict(sys.argv[1:],_i.dict))

        wandb.init(
            job_type    = _i.wandb.job_type,
            entity      = _i.wandb.entity,
            project     = _i.project,
            dir         = _i.exp_path,
            config      = dict_to_wandb(_i.dict),
            mode        = wandb_mode,
            settings=wandb.Settings(start_method='fork'), # idk y this is issue, don't change
        )

        if _i._submit_state > 0:
            n_job_running = run_cmds([f'squeue -u {_i.user} -h -t pending,running -r | wc -l'])
            if n_job_running > cap:
                exit(f'There are {n_job_running} on the submit cap is {cap}')

            _slurm = Slurm(**_i.slurm.dict)

            n_run, _i._submit_state = _i._submit_state, 0            
            for _ in range(n_run):
                _slurm.sbatch(_i.slurm.sbatch 
                + f'out_dir={(mkdir(_i.exp_path/"out"))} {_i.cmd} | tee $out_dir/py.out date "+%B %V %T.%3N" ')

    @property
    def cmd(_i,):
        d = flat_dict(_i.dict)
        return ' '.join([f' --{k}  {str(v)} ' for k,v in d.items()])

    @property
    def commit_id(_i,)->str:
        process = run_cmds(['git log --pretty=format:%h -n 1'], cwd=_i.project_path)[0]
        return process.stdout.decode('utf-8') 

    def submit(_i, sweep=False, commit_msg=None, cap=40):
        commit_msg = commit_msg or _i.exp_id
        _i._submit_state *= -1
        if _i._submit_state > 0:
            if sweep:
                _i.sweep_id = wandb.sweep(
                    env     = f'conda activate {_i.env};',
                    sweep   = _i.sweep.dict, 
                    program = _i.run_path,
                    project = _i.project,
                    name    = _i.exp_name,
                    run_cap = _i.sweep.n_sweep
                )
                _i._submit_state *= _i.sweep.n_sweep
            local_out = run_cmds(['git add .', f'git commit -m {commit_msg}', 'git push'], cwd=_i.project_path)
            server_out = run_cmds_server(_i.server, _i.user, f'python -u {_i.run_path} ' +_i.cmd, cwd=_i.server_project_path)

from pprint import pprint

c = Pyfig(wandb_mode='disabled', notebook=True)
pprint(c.dict)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'af': 'tanh',
 'commit_id': 'b78c668',
 'data': {'b_size': 16,
          'cache': False,
          'channels': 1,
          'dataset': 'fashion_mnist',
          'image_size': 28},
 'data_dir': PosixPath('/home/amawi/projects/data'),
 'dtype': 'f32',
 'entity': 'xmax1',
 'env': 'dex',
 'exp_id': 'N0X0xSI',
 'exp_name': 'junk',
 'exp_path': PosixPath('/home/amawi/projects/hwat/exp/junk/N0X0xSI'),
 'git_branch': 'main',
 'git_remote': 'origin',
 'half_precision': True,
 'iter_exp_dir': True,
 'log_metric_step': 5,
 'log_sample_step': 5,
 'log_state_step': 10,
 'model': {'dim': 64, 'dim_mults': (1, 2, 4)},
 'n_device': 1,
 'n_epoch': 20,
 'n_layer': 3,
 'n_step': 1000,
 'opt': {'beta1': 0.9,
         'beta2': 0.99,
         'eps': 1e-08,
         'loss': 'l1',
         'lr': 0.001,
         'optimizer': 'Adam'},
 'project': 'hwat',
 'project_cfg_dir': PosixPath('/home/amawi/projects/hwat/cfg'),
 'project_exp_dir': PosixPath('/home/amawi/projects/hwat/exp'),
 'project_path': PosixPath('/h

In [ ]:
def compute_metric(d:dict):
    ...
    metrics = lax.pmean(metrics, axis_name='b')

    return metric


def compute_energy():
    

    return 

In [ ]:

def compute_s_perm(x, x_p, p_mask_u, p_mask_d, n_u):
    n_e, _ = x_p.shape
    n_u_ish, _ = x.shape
    n_d = n_e - n_u_ish
    n_u = n_e - n_d

    xu, xd = jnp.split(x, [n_u, n_d], axis=0)
    mean_xu = jnp.mean(xu, axis=0, keepdims=True)
    mean_xd = jnp.mean(xd, axis=0, keepdims=True)

    x_p = jnp.expand_dims(x_p, axis=0)
    sum_p_u = (p_mask_u * x_p).sum((1, 2)) / float(n_u)
    sum_p_d = (p_mask_d * x_p).sum((1, 2)) / float(n_d)

    x = jnp.concatenate((x, mean_xu, mean_xd, sum_p_u, sum_p_d), axis=-1)
    return jnp.split(x, [n_u, n_d], axis=0)

def logabssumdet(orb_u, orb_d=None):
    # Special case if there is only one electron in any channel
    # We can avoid the log(0) issue by not going into the log domain
    
    xs = [orb_u, orb_d] if not orb_d is None else [orb_u]
    
    dets = [x.reshape(-1) for x in xs if x.shape[-1] == 1]
    dets = reduce(lambda a,b: a*b, dets) if len(dets)>0 else 1

    slogdets = [jnp.linalg.slogdet(x) for x in xs if x.shape[-1] > 1]
    
    if len(slogdets) > 0: # at least 2 electon in at least 1 orbital
        sign_in, logdet = reduce(lambda a,b: (a[0]*b[0], a[1]+b[1]), slogdets)
        maxlogdet = jnp.max(logdet)
        det = sign_in * dets * jnp.exp(logdet-maxlogdet)
    else:
        maxlogdet = 0
        det = dets

    psi_ish = jnp.sum(det)
    sgn_psi = jnp.sign(psi_ish)
    log_psi = jnp.log(jnp.abs(psi_ish)) + maxlogdet
    return log_psi, sgn_psi

def create_masks(n_electrons, n_up):
    ups = jnp.ones(n_electrons)
    ups[n_up:] = 0.
    downs = (ups-1.)*-1.

    pairwise_up_mask = []
    pairwise_down_mask = []
    for electron in range(n_electrons):
        mask_up = jnp.zeros((n_electrons, n_electrons))
        mask_up[electron, :] = ups
        pairwise_up_mask.append(mask_up)
        # mask_up = mask_up[eye_mask].reshape(-1) # for when drop diagonal enforced
        mask_down = jnp.zeros((n_electrons, n_electrons))
        mask_down[electron, :] = downs
        pairwise_down_mask.append(mask_down)

    pairwise_up_mask = jnp.stack(pairwise_up_mask, axis=0)[..., None]
    pairwise_down_mask = jnp.stack(pairwise_down_mask, axis=0)[..., None]
    return pairwise_up_mask, pairwise_down_mask

from typing import Callable

In [10]:
n_e = 10
x = jnp.ones((2, n_e, 3))
xu, xd = x.split(2, axis=1)

def init_walker(xu, xd):
    rng = rnd.PRNGKey(c.seed)
    rng_u, rng_d = rnd.split(rng, 2)
    return jnp.concatenate([rnd.normal(rng_u, xu.shape), rnd.normal(rng_d, xd.shape)], axis=1)



from jax import pmap

class SampleState():
    """
    @struct.dataclass
    struct.PyTreeNode means jax transformations *do not affect it* eg pmap
    fn.apply << apply vs fn() << apply_fn"""
    
    def __init__(_i, step=0, move_std=0.02, corr_len=20):
        _i.acc_target = 0.5
        _i.corr_len = corr_len//2
        _i.move_std = move_std
        _i.step = step
    
    def __call__(_i, x, state:TrainState):
        _i.rng, rng_0, rng_1, rng_move = rnd.split(_i.rng, 4)

        x, acc_0 = sample(rng_0, x, state, _i.corr_len, _i.move_std)
        move_std_1 = jnp.clip(_i.move_std + 0.001*rnd.normal(rng_move))
        x, acc_1 = sample(rng_1, x, state, _i.corr_len, move_std_1)

        mask = jnp.array((_i.acc_target-acc_0)**2 < (_i.acc_target-acc_1)**2, dtype=jnp.float32)
        not_mask = ((mask-1.)*-1.)
        _i.move_std = mask*_i.move_std + not_mask*move_std_1
        return x

def sample(rng, x, state:TrainState, corr_len, move_std):

    def move(x, rng, move_std):
        x = x + rnd.normal(rng, x.shape)*move_std
        return x

    to_prob = lambda log_psi: jnp.exp(log_psi)**2
    
    p = to_prob(state(x))
    
    acc = 0.0
    for _ in range(corr_len//2):
        rng, rng_move, rng_alpha = rnd.split(rng)
        
        x_1 = move(x, rng_move, move_std)
        p_1 = to_prob(state(x_1))

        p_mask = (p_1 / p) > rnd.uniform(rng_alpha, p_1.shape)
        p = jnp.where(p_mask, p_1, p)
        p_mask = jnp.expand_dims(p, axis=(-1, -2))
        x = jnp.where(p_mask, x_1, x)

        acc += jnp.mean(p_mask)

    return x, acc

class FermiNet(nn.Module):
  
  n_sv: int
  n_pv: int
  n_fb: int
  n_fb_out: int
  n_det: int

  af_fb: Callable
  af_fb_out: Callable
  af_pseudo: Callable

  compute_s_emb: Callable
  compute_p_emb: Callable
  compute_s_var: Callable
  compute_p_var: Callable
  compute_s_emb: Callable
  compute_p_emb: Callable
  
  n_e: int
  n_u: int
  n_d: int = n_e-n_u
  p_mask_u, p_mask_d = create_masks(n_e, n_u)
  compute_s: Callable = partial(compute_s, p_mask_u=p_mask_u, p_mask_d=p_mask_d, n_u=n_u)

  pbc:          bool = False

  @nn.compact
  def __call__(_i, x):
    
    n_e, n_u, n_d, n_det = _i.n_e, _i.n_u, _i.n_d, _i.n_det

    x_s_var = _i.compute_s_emb(x)
    x_p_var = _i.compute_p_emb(x)

    x_s_res = x_p_res = 0.
    for _ in range(_i.n_fb):
        x_p_var = x_p_res = _i.af_fb(nn.Dense(_i.n_pv)(x_p_var)) + x_p_res
        
        x_s_var = _i.compute_s(x_s_var, x_p_var)
        x_s_var = x_s_res = _i.af_fb(nn.Dense(_i.n_sv)(x_s_var)) + x_s_res

    x_w = jnp.concatentate([x_s_var, x_p_var], axis=-1)
    x_w = _i.af_fb_out(nn.Dense(_i.n_fb_out)(x_w))
    x_w = _i.af_pseudo(nn.Dense(n_det*n_e)(x_w))
    x_wu, x_wd = jnp.split(x_w, [n_u, n_d], axis=0)
    
    orb_u = jnp.split(x_wu * jnp.exp(-nn.Dense(n_u*n_det)), n_det, axis=1)
    orb_d = jnp.split(x_wd * jnp.exp(-nn.Dense(n_d*n_det)), n_det, axis=1)

    log_psi, sgn = logabssumdet(orb_u, orb_d)

    return log_psi


x = init_walker(xu, xd)

In [ ]:

# jax.tree_map(lambda x: x.shape, params) # Check the parameters





def create_train_state(c: Pyfig):
  model = CNN()
  params = model.init(c.rng_init, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(c.lr)
  return TrainState.create(apply_fn=model.apply, params=params, tx=tx)

state = create_train_state(c)
state = jax_utils.replicate(state)

@jax.jit
def train_step(params, state, b):
  
  b_energy = compute_energy(state)
  
  def loss_fn(p):
    model_out = state.apply_fn({'params':p}, b)
    log_psi, sgn = model_out
    return log_psi, sgn
  
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (log_psi, sgn), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  
  variables = {
    'E batch'           : b_energy,
    r'$\log\psi$ batch' : log_psi,
    r'sgn$(\cdot)$'     : sgn
  }
  
  metrics = compute_metric(b_energy, log_psi, sgn)
  return state, metrics

train_step = jax.pmap(train_step, axis_name='batch')

In [ ]:
def get_posterior_mean_variance(img, t, x0, v, ddpm):

    beta = ddpm_param.betas[t,None,None,None] # only needed when t > 0
    alpha = ddpm_param.alphas[t,None,None,None]
    alpha_bar = ddpm_param.alphas_bar[t,None,None,None]
    alpha_bar_last = ddpm_param.alphas_bar[t-1,None,None,None]
    sqrt_alpha_bar_last = ddpm_param.sqrt_alphas_bar[t-1,None,None,None]

    coef_x0 = beta * sqrt_alpha_bar_last / (1. - alpha_bar)
    coef_xt = (1. - alpha_bar_last) * jnp.sqrt(alpha) / ( 1- alpha_bar)        
    posterior_mean = coef_x0 * x0 + coef_xt * img
        
    posterior_variance = beta * (1 - alpha_bar_last) / (1. - alpha_bar)
    posterior_log_variance = jnp.log(jnp.clip(posterior_variance, a_min = 1e-20))

    return posterior_mean, posterior_log_variance


def ddpm_sample_step(state, rng, x, t, x0_last, ddpm_param: ddpm_param):
 
    batched_t = jnp.ones((x.shape[0],), dtype=jnp.int32) * t
    
    if c.ddpm.self_condition:
        x0, v = model_predict(state, x, x0_last, batched_t, ddpm_param, use_ema=True) 
    else:
        x0, v = model_predict(state, x, None, batched_t, ddpm_param, use_ema=True)
    
    x0 = jnp.clip(x0,-1.,1.) # make sure x0 between [-1,1]
    posterior_mean, posterior_log_variance = get_posterior_mean_variance(x, t, x0, v, ddpm_param)
    x = posterior_mean + jnp.exp(0.5 *  posterior_log_variance) * jax.random.normal(rng, x.shape) 
    return x, x0

sample_step = functools.partial(
    ddpm_sample_step, 
    ddpm_param=ddpm_param, 
    self_condition=c.ddpm.self_condition, 
    is_pred_x0=c.ddpm.is_pred_x0
)

p_sample_step = jax.pmap(sample_step, axis_name='batch')

In [ ]:
def copy_params_to_ema(state):
   state = state.replace(params_ema = state.params)
   return state

def apply_ema_decay(state, ema_decay):
    params_ema = jax.tree_map(lambda p_ema, p: p_ema * ema_decay + p * (1. - ema_decay), state.params_ema, state.params)
    state = state.replace(params_ema = params_ema)
    return state

p_apply_ema = jax.pmap(apply_ema_decay, in_axes=(0, None), axis_name='batch')
p_copy_params_to_ema = jax.pmap(copy_params_to_ema, axis_name='batch')


In [ ]:
from pprint import pprint
from pathlib import Path

def to_wandb_config(d, parent_key: str = '', sep: str ='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(to_wandb_config(v, new_key, sep=sep).items())
        else:
            if isinstance(v, Path):
                v = str(v)
            items.append((new_key, v))
    return dict(items)

wandb.init(
    job_type=c.wandb.job_type,
    entity=c.wandb.entity,
    project=c.project,
    config=to_wandb_config(c.dict),
    settings=wandb.Settings(start_method='fork'),  # idk why this is an issue
    dir=c.exp_path,
)

wandb.define_metric("*", step_metric="train/step")

# # Display a project workspace
# %wandb USERNAME/PROJECT
# # Display a single run
# %wandb USERNAME/PROJECT/runs/RUN_ID
# # Display a sweep
# %wandb USERNAME/PROJECT/sweeps/SWEEP_ID
# # Display a report
# %wandb USERNAME/PROJECT/reports/REPORT_ID
# # Specify the height of embedded iframe
# %wandb USERNAME/PROJECT -h 2048

In [ ]:
def get_data_stats():
    def normalize_to_neg_one_to_one(img):
        return img * 2 - 1  
    data_tr = datasets.FashionMNIST(
        root=c.data_dir, 
        download=True, 
        train=True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(28),
            transforms.CenterCrop(28),
        ]),
        target_transform = transforms.Compose([
        transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
    ])
    )
    imgs = torch.stack([img_t for img_t, _ in data_tr], dim=3)
    mean = imgs.view(1,-1).mean(dim=1) 
    std = imgs.view(1,-1).std(dim=1)
    print('Data mean: ', mean, 'Data std: ', std)
    return mean, std
mean, std = get_data_stats()
loader_tr, loader_test = get_fashion_loader(c.data.b_size, c.data_dir, mean=mean, std=std)  # is not an iterator or list
img, l = next(iter(loader_tr))

In [ ]:
import numpy as np
Image.fromarray(np.uint8((img[0, 0]*std+mean).cpu().numpy()*255))

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('1', size=(cols*w, rows*h))
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))

    return grid
std = float(std.cpu().numpy())
mean = float(mean.cpu().numpy())

In [ ]:
train_metrics = []

for ep in range(c.n_epoch):
    for step, (b, target) in enumerate(loader_tr):
        b = jnp.squeeze(jnp.expand_dims(jnp.array(b.numpy()), axis=(0,-1)), axis=2) # p, B, H, W, C
        rng, *train_step_rng = jax.random.split(rng, num=jax.local_device_count() + 1)
        train_step_rng = jnp.array(train_step_rng)
        
        state, metrics = p_train_step(train_step_rng, state, b)

        if step <= c.ema.update_after_step:
            state = p_copy_params_to_ema(state)

        elif step % c.ema.update_every == 0:
            ema_decay = ema_decay_fn(step)
            state =  p_apply_ema(state, ema_decay)

        if step % c.log_metric_step == 0:

            train_metrics.append(metrics)
            train_metrics = common_utils.get_metrics(train_metrics)
            
            summary = {
                f'train/{k}': v
                for k, v in jax.tree_map(lambda x: x.mean(), train_metrics).items()
            }

            train_metrics = []
            b = ((np.array(b)*std+mean)*255).astype(np.uint8)
            imgs = [wandb.Image(Image.fromarray(b[0, i].reshape(28, 28))) for i in range(9)]
            
            wandb.log({
                    "train/step": step, 
                    'train/sample': imgs,
                    **summary
            })
 
    print('Epoch: ', ep)

In [ ]:
# Image.fromarray(b[0, 0].reshape(28, 28, 1))
b[0, 0].max()